<h1> Convert json text to SQL DB </h1>

In [1]:
from __future__ import division
import psycopg2
import numpy as np
import json
import re


In [63]:
conn = psycopg2.connect("host=localhost port=5432 dbname=qa")
cur = conn.cursor()

<h1> Create SQL table to store the data </h1>

In [64]:
#QA db
cur.execute("CREATE TABLE product (id serial PRIMARY KEY,\
    Manufacturer text,\
    ProductGroup text,\
    Title text,\
    asin text\
    );")
conn.commit() #submit change to db

<h1> Load in all the product id </h1>

In [65]:
cur.execute("SELECT asin question from qa;")
ASIN=cur.fetchall()

In [66]:
unique_asin=set([asin[0] for asin in ASIN])

<h1> Loop through each asin and populate the DB </h1>

In [67]:
from amazonproduct import API
from amazonproduct.errors import AWSError
api = API(locale='us')

In [68]:
for asin in unique_asin:
    try:
        [item for item in api.item_search('All', Keywords=asin)]
        try:
            Manufacturer=item.ItemAttributes.Manufacturer
        except AttributeError:
            Manufacturer=''
        try:
            ProductGroup=item.ItemAttributes.ProductGroup
        except AttributeError:
            ProductGroup=''
        try:
            Title=item.ItemAttributes.Title
        except AttributeError:
            Title=''
    except AWSError:
        Manufacturer=''
        ProductGroup=''
        Title=''
        
    #insert into postgres
    cur.execute("INSERT INTO product (\
    Manufacturer,\
    ProductGroup,\
    Title,\
    asin\
    )\
    VALUES (\
    %s,%s,%s,%s\
    )",(u'{:s}'.format(Manufacturer),u'{:s}'.format(ProductGroup),u'{:s}'.format(Title),asin)\
    )
    conn.commit() #submit change to db

timeout: timed out